In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
#url = 'https://web.archive.org/web/20200720171507/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
# TODO - Wayback machine adds two extra tables at top we need to skip past
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [3]:
tables = soup.find_all('table')
#[len(a) for a in a[0].find_all('tr')]

In [4]:
#tables[2]

In [5]:
tmp = tables[0].find_all('caption')[0].get_text().split('/')
tmp[2]
#type(tmp)
#'-'.join(tmp.reverse())
reporting_date = tmp[2] + '-' + '%02d' % int(tmp[0]) + '-' + '%02d' % int(tmp[1])

In [6]:
print (reporting_date)
rows = tables[0].find_all('tr')
num_facilities = 0
for row in rows:
    if len(row) == 5:
        cells = row.find_all('td')
        if len(cells) == 2: # TODO = Should throw error if not
            print (cells[0].get_text(), cells[1].get_text())
            num_facilities += 1

2020 Statewide Report-11-02


In [7]:
cells = rows[2].find_all('td')
[a.get_text() for a in cells]

['181', '301', '60%', '0', '0', '0', '0', '725', '290', '534', '2']

In [8]:
def fix_MI_values(val):
    if val == '--':
        return np.nan
    if type(val) == int:
        return val
    else :
        return int(val.replace('*', '').replace('^', '').strip())

In [9]:
rows = tables[2].find_all('tr')
county="ERROR" # SHould throw exception and catch these, but for now
facilities = []
new_county = True
for row in rows:
    if len(row) == 11:
        cells = row.find_all('td')
        if new_county:
            headers = ['County'] + [cell.get_text() for cell in cells]
            headers[2] = 'Resident ' + headers[2]
            headers[3] = 'Resident ' + headers[3]
            headers[4] = 'Staff ' + headers[4]
            headers[5] = 'Staff ' + headers[5]
            #print(headers)
            new_county = False
        else:
            values = [cell.get_text() for cell in cells]
            #values = [values[0]] + [int(a.replace('*', '').replace('--', '-1').strip()) for a in values[1:]]
            values = [values[0]] + [fix_MI_values(a) for a in values[1:]]
            facilities.append(dict(zip(headers, [county] + values)))
            
    if len(row) == 7:
        cells = row.find_all('td')
        county=cells[0].get_text()
        new_county = True
        #print('County = ' + cells[0].get_text())

In [10]:
df = pd.DataFrame(facilities)

print("Num of Facilities:" + str(len(df)))

Num of Facilities:0


In [11]:
df = pd.DataFrame(facilities)
df_county = df.groupby(by=['County']).sum()
#TODO - add date to filename
#TODO - get date from scaping web site
df_county.sort_values('Resident Cumulative Confirmed Cases', ascending=False).to_csv('Reporting_data/MI_county_stats_' + reporting_date + '.csv')

KeyError: 'County'

In [ ]:
df.sort_values('Resident Cumulative Confirmed Cases', ascending=False).to_csv('Reporting_data/MI_' + reporting_date + '_LTC_data.csv', index=False)

In [ ]:
a='2*'
int(a.replace('*', '').replace('--', '-1').strip())